In [1]:
import numpy as np
from google.cloud import bigquery

In [5]:
PROJECT = "le-wagon-hedge-fund"
DATASET = "data_alpaca_20240604"
TABLE = "SP500_Historical_Weekly"

query = f'''
SELECT * FROM {PROJECT}.{DATASET}.{TABLE}
WHERE symbol = 'AAPL'
LIMIT 10
'''

client = bigquery.Client(project=PROJECT)
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()
df.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2016-01-04 05:00:00+00:00,102.61,105.85,96.43,96.96,362768447.0,1964372.0,100.620442
1,AAPL,2016-01-11 05:00:00+00:00,98.97,101.19,95.36,97.13,321021271.0,1746672.0,98.299269
2,AAPL,2016-01-18 05:00:00+00:00,98.41,101.46,93.42,101.42,254535461.0,1399429.0,97.214443
3,AAPL,2016-01-25 05:00:00+00:00,101.52,101.53,92.39,97.34,399618939.0,2037460.0,96.674642
4,AAPL,2016-02-01 05:00:00+00:00,96.47,97.33,93.69,94.02,226792333.0,1321006.0,95.583457


In [6]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns